# AD integration

## 0. Imports

In [1]:
import sys; sys.path.insert(0,"../..") # Allow import of agd from parent directory (useless if conda package installed)

In [2]:
from agd import HFMUtils
from agd import Metrics

In [3]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
def reload_packages():
    from importlib import reload
    HFMUtils = reload(sys.modules['agd.HFMUtils'])

In [5]:
reload_packages()

## 1. Basic problem

In [6]:
hfmIn = {}
hfmIn['model']='Isotropic2' # Alternatively 'IsotropicDiff2'

In [7]:
hfmIn['arrayOrdering']='RowMajor'
# Define the domain
hfmIn.update(
    HFMUtils.Rect(sides=[[-1,1],[0,1]],gridScale=1./100.)
)

# Set up the boundary conditions
hfmIn['seeds']=[[-0.5,0.3],[0.5,0.8]] # Seed position
hfmIn['seedValues']=[0.,0.5] # Boundary condition imposed at the seed. Defaults to $[0.,0.]$.

# Define the speed function
X,Y = HFMUtils.GetGrid(hfmIn) # Create a coordinate system

# Insert the obstacles
disk = (X-0.3)**2 + (Y-0.3)**2 <= 0.2**2
barrier = np.logical_and(X==X[0,70], Y>=0.4)
hfmIn['walls']= np.logical_or(disk,barrier) 

# Request the desired outputs
hfmIn['exportValues']=1. # Ask for the PDE solution
hfmIn['tips'] = [[0.,0.6],[-0.9,0.5],[0.8,0.8]] # Ask for the geodesics from these three points

In [8]:
hfmIn['cost'] = Metrics.Isotropic(np.exp(-0.5*(X**2+Y**2))) # Define the cost function

In [9]:
reload_packages()

In [10]:
hfmOut = HFMUtils.Run(hfmIn,smart=True)

Field verbosity defaults to 1
Field order defaults to 1
Field spreadSeeds defaults to -1
Field showProgress defaults to 0
Field factoringMethod defaults to None
Fast marching solver completed in 0.005003 s.
Field geodesicSolver defaults to Discrete
Field geodesicStep defaults to 0.25
Field geodesicWeightThreshold defaults to 0.001
Field geodesicVolumeBound defaults to 8.45
Field exportActiveNeighs defaults to 0
Field exportGeodesicFlow defaults to 0



## Forward AD

## Old

In [ ]:
from agd import HFMUtils

In [9]:
from agd import AutomaticDifferentiation as ad
import numpy as np

In [3]:
ad.array(((1,0),(0,1)))

array([[1, 0],
       [0, 1]])

In [7]:
u = ad.Dense.identity((2,))

In [10]:
ad.array((u,np.zeros((2,))))

denseAD(array([[0., 0.],
       [0., 0.]]),
array([[[1., 0.],
        [0., 1.]],

       [[0., 0.],
        [0., 0.]]]))

In [12]:
5 not in [2,3,4]

True

In [15]:
dico = {"a":2}
dico.get("b",3)

3

In [19]:
if 0.: print(True)

In [20]:
"abc" is "abc"

True

In [21]:
2. is 2.

True

In [22]:
2. is 2

False

In [23]:
"ab_"+str(2)

'ab_2'